<a href="https://colab.research.google.com/github/noambassat/SupremeCourtClassifier/blob/main/8_Statistics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [93]:
from google.colab import drive
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [94]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [95]:
file_path = "/content/drive/MyDrive/"
data_name  = "full_rap_df_cleaned_3_classifiers.xlsx"

In [ ]:
data = pd.read_excel(file_path + data_name)

# Classifier's Statistics

In [ ]:
def plot_statistics(data, col, quest):
    # ספירת המקרים השונים בעמודה
    value_counts = data[col].value_counts()
    print(value_counts)
    # יצירת הגרף
    fig, ax = plt.subplots(figsize=(8, 8))
    wedges, texts, autotexts = ax.pie(
    value_counts,
    labels=quest,
    autopct='%1.1f%%',
    textprops=dict(color="black"),
    startangle=90
)

# התאמת גודל הטקסטים
    plt.setp(autotexts, size=10, weight="bold")
    plt.setp(texts, size=12)
    plt.title(f'Distribution of {col}')

    plt.show()

    # הדפסת הנתונים מתחת לגרף
    print(f"Data distribution for '{col}':")

    total = sum(map(lambda count: count, value_counts))

    print(f"Total of {total} cases where checked.")

* Classifier "other cases" treshold for predction: 0.56. text cut : 400 - 30

* First classiier (requests) treshold : 0.67. text cut: 500 - 30

* Second classifier (appeals) treshold: 0.9. text cut: 550 - 80

---



In [ ]:
plot_statistics(data,'Others_classifier_treshold_prediction',['No','Classified as Others'])

In [ ]:
plot_statistics(data,'first_classifier_treshold_prediction',['Request Denied','Request Granted'])

In [ ]:
plot_statistics(data,'second_classifier_treshold_prediction',['Apeal itself Denied','Apeal itself Granted'])

# Add public and private sides

In [ ]:
sides_path = "sides_values with lables.xlsx"
sides_df = pd.read_excel(file_path+sides_path)

In [ ]:
sides_df["Public Sides"].unique()[5]

In [ ]:
sides_df["Private Sides"].unique()[5]

In [ ]:
def debug_message(message, value=None):
    if value is not None:
        print(f"{message}: {value}")
    else:
        print(message)

# פונקציה לניקוי ערכים
def safe_str(value):
    """
    Converts a value to a safe string, handling non-string types gracefully.
    """
    try:
        if pd.isna(value) or value == "" or value == []:
            return ""
        if isinstance(value, list):
            return " ".join([str(v) for v in value if isinstance(v, str)]).strip()
        return str(value).strip()
    except Exception as e:
        debug_message("Error converting value to string", value)
        return ""

# פונקציה לניקוי טקסט
def clean_text(text):
    """
    Cleans the text by removing unwanted characters and fixing formatting issues.
    """
    # Remove leading and trailing whitespace
    text = text.strip()

    # Fix \n or n between Hebrew words
    text = re.sub(r'(?<=[\u0590-\u05FF])\s*[n\\n]+\s*(?=[\u0590-\u05FF])', ' ', text)

    # Replace multiple spaces with a single space
    text = re.sub(r'\s+', ' ', text)

    return text

def publc_side(value, sides_df):
    """
    Determines if the value belongs to a public entity based on predefined lists and patterns.
    """
    try:
        value = safe_str(value)
        value_cleaned = clean_text(value)

        # בדיקת מילים קבועות בתור "ציבורי"
        keywords = sides_df["Public Sides"].dropna().unique().tolist()
        public_terms = [
        "רשות מקרקעי ישראל", "מדינת ", "מועצה ", "שירות ה", "היחידה הארצי", "מדינתישראל", "לאכיפת ", "הרשות ל", "חוף ",
        "המשרד ל", 'יועמ"ש', "המשפטי לממשלה", "מנהל מקרקעי", "פרקליטות", "שירות בתי הסוהר", 'שב"ס', 'מע"מ', "מכס ", 'קמ"ט',
        "ועדת תכנון", "מרכז השלטון", "משטרה", "הסנגוריה הציבורית", "מינהל מקרקעי", "עירית ", "רשויות", "איגוד ערים",
        "עיריית ", "מדינת ישראל", "משרד ה", "מקרקעי ישראל", "שר ה", "שרת ה", "מועצת ", 'רנ"ג', "סרן", "מחוז ","מחש",
        "מ דינת ישראל", "שרות בתי הסוהר", 'סג"ם', "(במיל)", "(מיל)", "עריית ", "המחלקה לחקירת שוטרים", "יושבי ראש",'מח"ש'
        "רשות ה", "אלוף ", 'יו"ר', "יושב ראש", "ראש עיריית", "ועדה", "צבא", "מפקד", "פיקוד", "משטרת", "משטרה",'רנ"ג',"המחלקה לחקירות שוטרים"
    ]
        # אם אחד מהמונחים מופיע, צד ציבורי
        if any(term in value_cleaned for term in public_terms + keywords):
            return 1

        # במידה ואין התאמה, נבדוק לפי אורכים ודפוסים כלליים
        if len(value_cleaned) > 5 and " " in value_cleaned:
            return 0

        return None
    except Exception as e:
        return None


def assign_state_side_b(value, sides_df):
    """
    Maps the side to public ('ב'), private ('א'), or undefined ('לא מוגדר').
    """
    try:
        value = safe_str(value)
        result = publc_side(value, sides_df)

        if result == 1:
            return "ב"
        elif result == 0:
            return None
        else:
            return "לא מוגדר"
    except Exception as e:
        debug_message("Error assigning state side for value", value)
        return "לא מוגדר"

def assign_state_side_a(value, sides_df):
    """
    Maps the side to public ('ב'), private ('א'), or undefined ('לא מוגדר').
    """
    try:
        value = safe_str(value)
        result = publc_side(value, sides_df)

        if result == 1:
            return "א"
        elif result == 0:
            return None
        else:
            return "לא מוגדר"
    except Exception as e:
        debug_message("Error assigning state side for value", value)
        return "לא מוגדר"

In [ ]:
check_nulls_b = data[data["צד ב'"].isna()]
check_nulls_b["צד א'"].unique()

In [ ]:
# ניקוי כללי של העמודה "צד ב'"
data["צד ב'"] = data["צד ב'"].astype(str).str.strip()  # הסרת רווחים
data["צד ב'"] = data["צד ב'"].replace(r'\s+', ' ', regex=True)  # הסרת רווחים מיותרים
data["צד ב'"] = data["צד ב'"].replace(r'[^\w\s]', '', regex=True)  # הסרת תווים מיוחדים

In [ ]:
data["צד המדינה"] = data["צד ב'"].apply(lambda x: assign_state_side_b(x, sides_df))
data["צד המדינה"] = data.apply(
    lambda row: assign_state_side_a(row["צד א'"], sides_df) if row["צד המדינה"] in ["לא מוגדר", ""] else row["צד המדינה"],
    axis=1
)

In [ ]:

unknown_values = data[data["צד המדינה"] == "לא מוגדר"]
unknown_values["צד ב'"].unique()


In [ ]:
unknown_values["צד א'"].unique()

In [ ]:
# plot_statistics(data,'צד המדינה',['א','ב',"None"])

In [ ]:
data.to_excel(file_path + "בית המשפט - קבצים סופיים/full_rap_df_cleaned_3_classifiers_with_state's_side.xlsx", index=False)

# הוספת צד מדינה לכל הדאטה

In [ ]:
all_data = pd.read_excel(file_path+
    "full_rap_df_cleaned.xlsx",
    engine='openpyxl'
)

In [ ]:
all_data["צד ב'"] = data["צד ב'"].astype(str).str.strip()  # הסרת רווחים
all_data["צד ב'"] = data["צד ב'"].replace(r'\s+', ' ', regex=True)  # הסרת רווחים מיותרים
all_data["צד ב'"] = data["צד ב'"].replace(r'[^\w\s]', '', regex=True)  # הסרת תווים מיוחדים

all_data["צד המדינה"] = all_data["צד ב'"].apply(lambda x: assign_state_side_b(x, sides_df))
all_data["צד המדינה"] = all_data.apply(
    lambda row: assign_state_side_a(row["צד א'"], sides_df) if row["צד המדינה"] in ["לא מוגדר", ""] else row["צד המדינה"],
    axis=1
)

In [ ]:
all_data.to_excel("/content/drive/MyDrive/בית המשפט - קבצים סופיים/full_rap_df_cleaned_with_state_side.xlsx")

In [ ]:
all_data[all_data["צד המדינה"]=="לא מוגדר"]["צד ב'"].head()

# סטטיסטיקה עבור צד המדינה לכל תיקים הרע"פ

In [ ]:
plot_statistics(all_data,'צד המדינה',['ב','None',"א"])

In [ ]:
path = "/content/drive/MyDrive/בית המשפט - קבצים סופיים"

unknown_values = all_data[all_data["צד המדינה"] == "לא מוגדר"]

unknown_values.to_excel(path + "/תיקים בהם צד המדינה לא הוגדר.xlsx", index=False)

In [ ]:
unknown_values["צד א'"].unique()[1:5]

In [ ]:
only_known_sides = all_data[all_data["צד המדינה"] != "לא מוגדר"]

In [ ]:
only_known_sides[only_known_sides["צד המדינה"]=="א"]["צד ב'"].unique() # private

In [ ]:
only_known_sides[only_known_sides["צד המדינה"]=="ב"]["צד א'"].unique() # פרטי

In [ ]:
only_known_sides[only_known_sides["צד המדינה"]=="ב"]["צד ב'"].unique() # מדיני


In [ ]:
only_known_sides[only_known_sides["צד המדינה"]=="א"]["צד א'"].unique() # מדיני

In [ ]:
only_known_sides[only_known_sides["שנת פתיחת ההליך"]=="2002"]

In [ ]:
only_known_sides[only_known_sides["שנת פתיחת ההליך"]==2002]

In [ ]:


# # יצירת DataFrame עם איזון אורכים
# max_length = max(len(public_side), len(private_side))
# public_side.extend([None] * (max_length - len(public_side)))
# private_side.extend([None] * (max_length - len(private_side)))

# # יצירת הטבלה
# sides_Public_Private = {"Public Side": public_side, "Private Side": private_side}
# sides_Public_Private = pd.DataFrame(sides_Public_Private)


# sides_Public_Private.drop_duplicates(inplace=True)
# path = "/content/drive/MyDrive/בית המשפט - קבצים סופיים"
# output_path = path + "/sides_Public_Private.xlsx"
# sides_Public_Private.to_excel(output_path, index=False)